In [ ]:
# Ändra denna till rätt sökväg:
datafile = 'dbfs:/FileStore/shared_uploads/bedo.kharboutli@hotmail.com/kddcup_data_10_percent_corrected__1_-1.gz'

# Ladda in filen och gör den sökbar genom en DataFrame och via SQL-syntax
data = spark.read.format("csv").option("header", "true").load(datafile)
data.createOrReplaceTempView('data')

In [ ]:
# Räkna antalet rader i datan
data.count()

494021

In [ ]:
# Visa de första 20 raderna
data.show(20)

+--------+-------------+-------+----+---------+---------+----+--------------+------+---+-----------------+---------+----------------+-----------+-------------+---------+-------------------+-----------+-----------------+------------------+-------------+--------------+-----+---------+-----------+---------------+-----------+---------------+-------------+-------------+------------------+--------------+------------------+----------------------+----------------------+---------------------------+---------------------------+--------------------+------------------------+--------------------+------------------------+-------+
|duration|protocol_type|service|flag|src_bytes|dst_bytes|land|wrong_fragment|urgent|hot|num_failed_logins|logged_in|lnum_compromised|lroot_shell|lsu_attempted|lnum_root|lnum_file_creations|lnum_shells|lnum_access_files|lnum_outbound_cmds|is_host_login|is_guest_login|count|srv_count|serror_rate|srv_serror_rate|rerror_rate|srv_rerror_rate|same_srv_rate|diff_srv_rate|srv_diff_h

In [ ]:
# Här är ett exempel på en sökning i datan
interactions = spark.sql("""
SELECT duration, dst_bytes FROM data WHERE protocol_type = 'tcp' AND duration > 1000 AND dst_bytes = 0
""")
interactions.show()

+--------+---------+
|duration|dst_bytes|
+--------+---------+
|    5057|        0|
|    5059|        0|
|    5051|        0|
|    5056|        0|
|    5051|        0|
|    5039|        0|
|    5062|        0|
|    5041|        0|
|    5056|        0|
|    5064|        0|
|    5043|        0|
|    5061|        0|
|    5049|        0|
|    5061|        0|
|    5048|        0|
|    5047|        0|
|    5044|        0|
|    5063|        0|
|    5068|        0|
|    5062|        0|
+--------+---------+
only showing top 20 rows



In [ ]:
# Vi kan använda printSchema() för att visa vilka kolumner det finns
data.printSchema()

root
 |-- duration: string (nullable = true)
 |-- protocol_type: string (nullable = true)
 |-- service: string (nullable = true)
 |-- flag: string (nullable = true)
 |-- src_bytes: string (nullable = true)
 |-- dst_bytes: string (nullable = true)
 |-- land: string (nullable = true)
 |-- wrong_fragment: string (nullable = true)
 |-- urgent: string (nullable = true)
 |-- hot: string (nullable = true)
 |-- num_failed_logins: string (nullable = true)
 |-- logged_in: string (nullable = true)
 |-- lnum_compromised: string (nullable = true)
 |-- lroot_shell: string (nullable = true)
 |-- lsu_attempted: string (nullable = true)
 |-- lnum_root: string (nullable = true)
 |-- lnum_file_creations: string (nullable = true)
 |-- lnum_shells: string (nullable = true)
 |-- lnum_access_files: string (nullable = true)
 |-- lnum_outbound_cmds: string (nullable = true)
 |-- is_host_login: string (nullable = true)
 |-- is_guest_login: string (nullable = true)
 |-- count: string (nullable = true)
 |-- srv_c

In [ ]:
# Exempel på gruppering av data (Spark-syntax, ej SQL-syntax)
data.select("protocol_type", "duration", "dst_bytes").groupBy("protocol_type").count().show()

+-------------+------+
|protocol_type| count|
+-------------+------+
|          tcp|190065|
|          udp| 20354|
|         icmp|283602|
+-------------+------+



In [ ]:
data.select("protocol_type", "duration", "dst_bytes").filter(data.duration>1000).filter(data.dst_bytes==0).groupBy("protocol_type").count().show()

+-------------+-----+
|protocol_type|count|
+-------------+-----+
|          tcp|  139|
+-------------+-----+



## Answering the questions


In [ ]:
# Kod kod kod
# 1. Vilka protokoll finns representerade i datan?
protocols = spark.sql("SELECT DISTINCT protocol_type FROM data").collect()
print("Protokoll som finns representerade i datan:")
for protocol in protocols:
    print(protocol[0])

Protokoll som finns representerade i datan:
tcp
udp
icmp


In [ ]:
# 2. Hur många tjänster (services) är det som använder protkollet...
# TCP
tcp_services_count = spark.sql("SELECT COUNT(DISTINCT service) AS tcp_count FROM data WHERE protocol_type = 'tcp'").collect()[0]['tcp_count']
print("Antal tjänster som använder TCP-protokollet:", tcp_services_count)

# UDP
udp_services_count = spark.sql("SELECT COUNT(DISTINCT service) AS udp_count FROM data WHERE protocol_type = 'udp'").collect()[0]['udp_count']
print("Antal tjänster som använder UDP-protokollet:", udp_services_count)

# ICMP
icmp_services_count = spark.sql("SELECT COUNT(DISTINCT service) AS icmp_count FROM data WHERE protocol_type = 'icmp'").collect()[0]['icmp_count']
print("Antal tjänster som använder ICMP-protokollet:", icmp_services_count)


Antal tjänster som använder TCP-protokollet: 57
Antal tjänster som använder UDP-protokollet: 5
Antal tjänster som använder ICMP-protokollet: 6


In [ ]:
# 3.1. Hur många paket är klassade som en del av en attack, och använder protkollet…

# TCP
attack_tcp_count = spark.sql("SELECT COUNT(*) AS attack_tcp_count FROM data WHERE protocol_type = 'tcp' AND label != 'normal.'").collect()[0]['attack_tcp_count']
print("Antal attacker som använder TCP-protokollet:", attack_tcp_count)

# UDP
attack_udp_count = spark.sql("SELECT COUNT(*) AS attack_udp_count FROM data WHERE protocol_type = 'udp' AND label != 'normal.'").collect()[0]['attack_udp_count']
print("Antal attacker som använder UDP-protokollet:", attack_udp_count)

# ICMP
attack_icmp_count = spark.sql("SELECT COUNT(*) AS attack_icmp_count FROM data WHERE protocol_type = 'icmp' AND label != 'normal.'").collect()[0]['attack_icmp_count']
print("Antal attacker som använder ICMP-protokollet:", attack_icmp_count)

Antal attacker som använder TCP-protokollet: 113252
Antal attacker som använder UDP-protokollet: 1177
Antal attacker som använder ICMP-protokollet: 282314


In [ ]:
# 3.2. Hur många procent av det totala antalet paket (avrundat till hela procent) har en label som inte är ”normal.”?

total_packets = spark.sql("SELECT COUNT(*) AS total_packets FROM data").collect()[0]['total_packets']
attack_packets = spark.sql("SELECT COUNT(*) AS attack_packets FROM data WHERE label != 'normal.'").collect()[0]['attack_packets']
attack_percentage = (attack_packets / total_packets) * 100
print(round(attack_percentage, 2), "%")


80.31 %


In [ ]:
# 3.3. Hur många ICMP-paket är klassade som en del av en attack, men är inte en så kallad ”smurf”-attack?

icmp_attack_not_smurf_count = spark.sql("SELECT COUNT(*) AS icmp_attack_not_smurf_count FROM data WHERE protocol_type = 'icmp' AND label != 'normal.' AND service != 'smurf'").collect()[0]['icmp_attack_not_smurf_count']
print("Antal ICMP-paket som är klassade som en del av en attack, men inte en 'smurf'-attack:", icmp_attack_not_smurf_count)

Antal ICMP-paket som är klassade som en del av en attack, men inte en 'smurf'-attack: 282314
